In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import glob
import os
import time
import json
import copy
import random
import cv2
from IPython.display import display

In [2]:
plt.style.use('seaborn')
warnings.filterwarnings('ignore')

In [3]:
# File參數:
root = r'D:\桌面\基於深度學習之視覺辨識專論'
trainPath = os.path.join(root, 'training_data')
testPath = os.path.join(root, 'testing_data')
labelPath = os.path.join(root, 'training_labels.csv')

In [4]:
trainImgPaths = pd.DataFrame(glob.glob(os.path.join(trainPath, '*.jpg')), columns = ['path'])
testImgPaths = pd.DataFrame(glob.glob(os.path.join(testPath, '*.jpg')), columns = ['path'])
labels = pd.read_csv(labelPath)

print('Training images paths:')
display(trainImgPaths)
print('Testing images paths:')
display(testImgPaths)
print('Labels of images:')
display(labels)


Training images paths:


,path
0,D:\桌面\基於深度學習之視覺辨識專論\training_data\000001.jpg
1,D:\桌面\基於深度學習之視覺辨識專論\training_data\000002.jpg
2,D:\桌面\基於深度學習之視覺辨識專論\training_data\000003.jpg
3,D:\桌面\基於深度學習之視覺辨識專論\training_data\000007.jpg
4,D:\桌面\基於深度學習之視覺辨識專論\training_data\000009.jpg
...,...
11180,D:\桌面\基於深度學習之視覺辨識專論\training_data\016179.jpg
11181,D:\桌面\基於深度學習之視覺辨識專論\training_data\016182.jpg
11182,D:\桌面\基於深度學習之視覺辨識專論\training_data\016183.jpg
11183,D:\桌面\基於深度學習之視覺辨識專論\training_data\016184.jpg


Testing images paths:


,path
0,D:\桌面\基於深度學習之視覺辨識專論\testing_data\000004.jpg
1,D:\桌面\基於深度學習之視覺辨識專論\testing_data\000005.jpg
2,D:\桌面\基於深度學習之視覺辨識專論\testing_data\000006.jpg
3,D:\桌面\基於深度學習之視覺辨識專論\testing_data\000008.jpg
4,D:\桌面\基於深度學習之視覺辨識專論\testing_data\000019.jpg
...,...
4995,D:\桌面\基於深度學習之視覺辨識專論\testing_data\016172.jpg
4996,D:\桌面\基於深度學習之視覺辨識專論\testing_data\016175.jpg
4997,D:\桌面\基於深度學習之視覺辨識專論\testing_data\016176.jpg
4998,D:\桌面\基於深度學習之視覺辨識專論\testing_data\016180.jpg


Labels of images:


,id,label
0,9350,Ford F-150 Regular Cab 2007
1,2645,BMW X6 SUV 2012
2,2267,BMW 1 Series Coupe 2012
3,8553,Fisker Karma Sedan 2012
4,6990,Dodge Ram Pickup 3500 Crew Cab 2010
...,...,...
11180,184,Acura TL Sedan 2012
11181,5863,Chevrolet Malibu Sedan 2007
11182,2482,BMW 6 Series Convertible 2007
11183,14926,Suzuki Kizashi Sedan 2012


In [5]:
trainImgPaths['id'] = trainImgPaths['path'].apply(lambda x: int(os.path.split(x)[-1].replace('.jpg', '')))
trainImgPaths = pd.merge(trainImgPaths, labels, on='id')
trainImgPaths = trainImgPaths.drop('id', axis=1)

types = trainImgPaths['label'].unique()
nValid = round(trainImgPaths.shape[0]*0.1)
validImgPaths = {'path': [], 'label': []}
i = 0
while i < nValid:
    typesCopy = copy.copy(types)
    random.shuffle(typesCopy)
    for type_ in typesCopy:
        typePaths = trainImgPaths[trainImgPaths['label'] == type_]
        index = random.choice(typePaths.index)
        path = typePaths.loc[index, 'path']
        trainImgPaths.drop(index, inplace=True)
        validImgPaths['path'].append(path)
        validImgPaths['label'].append(type_)
        i += 1
        if i >= nValid: 
            break

validImgPaths = pd.DataFrame(validImgPaths)
trainImgPaths = trainImgPaths.reset_index(drop=True)

validImgPaths.to_csv(os.path.join(root, 'validImgPaths.csv'), index=False)
trainImgPaths.to_csv(os.path.join(root, 'trainImgPaths.csv'), index=False)

print('trainImgPaths: ')
display(trainImgPaths)
print('validImgPaths: ')
display(validImgPaths)



trainImgPaths: 


,path,label
0,D:\桌面\基於深度學習之視覺辨識專論\training_data\000001.jpg,AM General Hummer SUV 2000
1,D:\桌面\基於深度學習之視覺辨識專論\training_data\000002.jpg,AM General Hummer SUV 2000
2,D:\桌面\基於深度學習之視覺辨識專論\training_data\000003.jpg,AM General Hummer SUV 2000
3,D:\桌面\基於深度學習之視覺辨識專論\training_data\000007.jpg,AM General Hummer SUV 2000
4,D:\桌面\基於深度學習之視覺辨識專論\training_data\000009.jpg,AM General Hummer SUV 2000
...,...,...
10062,D:\桌面\基於深度學習之視覺辨識專論\training_data\016179.jpg,smart fortwo Convertible 2012
10063,D:\桌面\基於深度學習之視覺辨識專論\training_data\016182.jpg,smart fortwo Convertible 2012
10064,D:\桌面\基於深度學習之視覺辨識專論\training_data\016183.jpg,smart fortwo Convertible 2012
10065,D:\桌面\基於深度學習之視覺辨識專論\training_data\016184.jpg,smart fortwo Convertible 2012


validImgPaths: 


,path,label
0,D:\桌面\基於深度學習之視覺辨識專論\training_data\000999.jpg,Audi A5 Coupe 2012
1,D:\桌面\基於深度學習之視覺辨識專論\training_data\012720.jpg,Land Rover LR2 SUV 2012
2,D:\桌面\基於深度學習之視覺辨識專論\training_data\005061.jpg,Chevrolet Sonic Sedan 2012
3,D:\桌面\基於深度學習之視覺辨識專論\training_data\015619.jpg,Volkswagen Golf Hatchback 2012
4,D:\桌面\基於深度學習之視覺辨識專論\training_data\015810.jpg,Volkswagen Beetle Hatchback 2012
...,...,...
1113,D:\桌面\基於深度學習之視覺辨識專論\training_data\002247.jpg,BMW 1 Series Coupe 2012
1114,D:\桌面\基於深度學習之視覺辨識專論\training_data\016026.jpg,Volvo XC90 SUV 2007
1115,D:\桌面\基於深度學習之視覺辨識專論\training_data\006859.jpg,Dodge Caliber Wagon 2007
1116,D:\桌面\基於深度學習之視覺辨識專論\training_data\007778.jpg,Dodge Durango SUV 2007
